# Building a Euribor Curve - Interpolation Analysis - Simple Case

We build an Ibor single-curve and examine different interpolation choices for a simple curve.

This follows:

https://quant.stackexchange.com/questions/53904/monotonic-cubic-spline-interpolation-quantlib-python?rq=1

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from financepy.utils import *
from financepy.products.rates import *

In [ ]:
valuation_date = Date(12, 5, 2020)

## Building the Single Curve

Build the Ibor Curve

In [ ]:
cal = CalendarTypes.TARGET

depoDCCType = DayCountTypes.ACT_360
depos = []
spot_days = 0
settlement_date = valuation_date.add_weekdays(spot_days)
depo = IborDeposit(settlement_date, "6M", -0.114/100.0, depoDCCType, cal_type=cal); depos.append(depo)
fraDCCType = DayCountTypes.ACT_360

fras = []
fra = IborFRA(settlement_date.add_tenor("6M"), "6M", -0.252/100.0, fraDCCType, cal_type=cal); fras.append(fra)
fra = IborFRA(settlement_date.add_tenor("12M"), "6M", -0.306/100.0, fraDCCType, cal_type=cal); fras.append(fra)

swaps = []
swapType = SwapTypes.PAY
fixedDCCType = DayCountTypes.THIRTY_E_360_ISDA
fixedFreqType = FrequencyTypes.ANNUAL

swap = IborSwap(settlement_date, "2Y", swapType, -0.325/100.0, fixedFreqType, fixedDCCType, cal_type=cal); swaps.append(swap)
swap = IborSwap(settlement_date, "3Y", swapType, -0.347/100.0, fixedFreqType, fixedDCCType, cal_type=cal); swaps.append(swap)

ibor_depos = depos.copy()
ibor_fras = fras.copy()
ibor_swaps = swaps.copy()

In [ ]:
check_refit = True
ibor_curves = {}
for interp_type in InterpTypes:
# for interp_type in [InterpTypes.NATCUBIC_LOG_DISCOUNT]:
    optional_interp_params = {'sigma' : 20} 
    ibor_curve = IborSingleCurve(valuation_date, ibor_depos, ibor_fras, ibor_swaps, interp_type, check_refit=check_refit, **optional_interp_params)
    ibor_curves[interp_type] = ibor_curve

Set up a list of dates

In [ ]:
grid_dates = []
dt = Date(12, 5, 2020); grid_dates.append(dt)
dt = dt.add_tenor("6M"); grid_dates.append(dt)
dt = dt.add_tenor("6M"); grid_dates.append(dt)
dt = dt.add_tenor("6M"); grid_dates.append(dt)
dt = dt.add_tenor("6M"); grid_dates.append(dt)
dt = dt.add_tenor("12M"); grid_dates.append(dt)

In [ ]:
print("              ", end="")

for interp in ibor_curves.keys():
    print("%13s "% interp.name[0:15], end="")
print("")
    
for dt in grid_dates:
    print("%12s"% dt.str(), end="")
    for interp in ibor_curves.keys():
        ibor_curve = ibor_curves[interp]
        df = ibor_curve.df(dt) / ibor_curve.df(settlement_date)
        print("%15.7f "% (df), end="")
    print("")

This agrees perfectly with QL.

Now shift the dates and you see the different impact of the different interpolation schemes.

In [ ]:
shifted_dates = []
for dt in grid_dates:
    shifted_dates.append(dt.add_tenor("15D"))

In [ ]:
print("              ", end="")

for interp in ibor_curves.keys():
    print("%8s "% interp.name[0:13], end="")
print("")
    
for dt in shifted_dates:
    print("%8s "% dt.str(), end="")
    for interp in ibor_curves.keys():
        ibor_curve = ibor_curves[interp]
        df = ibor_curve.df(dt) / ibor_curve.df(settlement_date)
        print("%12.6f "% (df), end="")
    print("")

This shows that the interpolations do not match once we are off the grid times.

## Curve Shape Analysis

Look at short end

In [ ]:
years = np.linspace(0.1, 3, 1000)
plotDates = settlement_date.add_years(years)

for interp_type in ibor_curves.keys():

    plt.figure(figsize=(8,5))
    ibor_curve = ibor_curves[interp_type]
    iborCurveFwdRate = ibor_curve.fwd_rate(plotDates, "1D")
    iborCurveZeroRate = ibor_curve.zero_rate(plotDates)
    iborCurveSwapRate = ibor_curve.swap_rate(settlement_date, plotDates)
    
    plt.plot(years, iborCurveFwdRate*100.0, label="ON FWD RATE")
    plt.plot(years, iborCurveZeroRate*100.0, label="ZERO RATE")
    plt.plot(years, iborCurveSwapRate*100.0, label="SWAP RATE")

    plt.title(interp_type.name)
    plt.xlabel("Years")
    plt.ylabel("Rate (%)")
    plt.legend()

Copyright (c) Dominic O'Kane 2020